- Event
- Lock/RLock
- Condition
- Threading/Timer

## Barrier

In [6]:
import threading
import logging
import time
import importlib
importlib.reload(logging)
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(levelname)s %(threadName)s %(message)s')

In [2]:
def worker(barrier: threading.Barrier):
    logging.info('waiting for {} threading'.format(barrier.n_waiting))
    try:
        worker_id = barrier.wait()
    except threading.BrokenBarrierError:
        logging.warning('aborting')
    else:
        logging.info('after barrier {}'.format(worker_id))

In [3]:
barrier = threading.Barrier(3)

In [4]:
for x in range(3):
    threading.Thread(target=worker, name='worker-{}'.format(x), args=(barrier, )).start()
logging.info('started')

2017-11-12 09:13:34,801 INFO worker-0 waiting for 0 threading
2017-11-12 09:13:34,802 INFO worker-1 waiting for 0 threading
2017-11-12 09:13:34,802 INFO worker-2 waiting for 0 threading
2017-11-12 09:13:34,802 INFO MainThread started
2017-11-12 09:13:34,805 INFO worker-2 after barrier 2
2017-11-12 09:13:34,805 INFO worker-1 after barrier 1
2017-11-12 09:13:34,805 INFO worker-0 after barrier 0


In [5]:
def worker(barrier: threading.Barrier):
    logging.info('waiting for {} threading'.format(barrier.n_waiting))
    try:
        worker_id = barrier.wait()
        logging.info('after barrier {}'.format(worker_id))
    except threading.BrokenBarrierError:
        logging.warning('aborting')
        

In [7]:
barrier = threading.Barrier(3)

for x in range(3):
    threading.Thread(target=worker, name='worker-{}'.format(x), args=(barrier, )).start()
    time.sleep(0.5)
logging.info('started')

2017-11-12 09:20:54,100 INFO worker-0 waiting for 0 threading
2017-11-12 09:20:54,606 INFO worker-1 waiting for 1 threading
2017-11-12 09:20:55,109 INFO worker-2 waiting for 2 threading
2017-11-12 09:20:55,110 INFO worker-2 after barrier 2
2017-11-12 09:20:55,110 INFO worker-0 after barrier 0
2017-11-12 09:20:55,111 INFO worker-1 after barrier 1
2017-11-12 09:20:55,610 INFO MainThread started


In [14]:
barrier = threading.Barrier(3)

In [9]:
def abort(barrier):
    logging.info('aborting')
    barrier.abort()

In [10]:
for x in range(2):
    threading.Thread(target=worker, args=(barrier, ),  name='worker-{}'.format(x)).start()

2017-11-12 09:24:40,094 INFO worker-0 waiting for 0 threading
2017-11-12 09:24:40,094 INFO worker-1 waiting for 0 threading


In [11]:
barrier.abort()

2017-11-12 09:25:17,842 WARNING worker-1 aborting
2017-11-12 09:25:17,842 WARNING worker-0 aborting


In [16]:
barrier.n_waiting

2

In [15]:
for x in range(2):
    threading.Thread(target=worker, args=(barrier, ),  name='worker-{}'.format(x)).start()

2017-11-12 09:26:46,901 INFO worker-0 waiting for 0 threading
2017-11-12 09:26:46,902 INFO worker-1 waiting for 0 threading


In [17]:
barrier.abort()

2017-11-12 09:27:47,468 WARNING worker-0 aborting
2017-11-12 09:27:47,468 WARNING worker-1 aborting


barrier 在wait的时候，使用abort 方法会触发 exception: BrokenBarrierError

In [18]:
help(barrier.wait)

Help on method wait in module threading:

wait(timeout=None) method of threading.Barrier instance
    Wait for the barrier.
    
    When the specified number of threads have started waiting, they are all
    simultaneously awoken. If an 'action' was provided for the barrier, one
    of the threads will have executed that callback prior to returning.
    Returns an individual index number from 0 to 'parties-1'.



In [19]:
barrier.wait(3)

BrokenBarrierError: 

In [20]:
barrier = threading.Barrier(3)

In [21]:
barrier.wait(3)

BrokenBarrierError: 

## Semaphore

In [22]:
s = threading.Semaphore(3)

In [23]:
s.acquire()

True

In [24]:
s.acquire(False)

True

In [25]:
s.acquire(False)

True

In [26]:
s.acquire(False)

False

锁是一种特化的信号量

锁其实就是信号量为1的 特列

In [27]:
class Pool:
    def __init__(self, num):
        self.num = num
        
    def _make_connect(self):
        pass
    
    def get(self):
        pass
    
    def return_resource(self):
        pass

In [29]:
class Pool:
    def __init__(self, num):
        self.num = num
        self.conns = [self._make_connect() for x in range(num)]
        
    def _make_connect(self, name):
        return name
    
    def get(self):
        return self.conns.pop()
    
    def return_resource(self, conn):
        self.conns.insert(0, conn)

In [39]:
class Pool:
    def __init__(self, num):
        self.num = num
        self.conns = [self._make_connect(x) for x in range(num)]
        self.s = threading.Semaphore(num)
        
    def _make_connect(self, name):
        return name
    
    def get(self):
        self.s.acquire()
        return self.conns.pop()
    
    def return_resource(self, conn):
        self.conns.insert(0, conn)
        self.s.release()

In [40]:
import random

In [41]:
def worker(pool):
    logging.info('started')
    name = pool.get()
    logging.info('got connect {}'.format(name))
    time.sleep(random.randint(1, 3))
    pool.return_resource(name)
    logging.info('return resource {}'.format(name))

In [42]:
pool = Pool(3)

In [43]:
for x in range(5):
    threading.Thread(target=worker, args=(pool, ), name='worker-{}'.format(x)).start()

2017-11-12 10:19:50,476 INFO worker-0 started
2017-11-12 10:19:50,477 INFO worker-1 started
2017-11-12 10:19:50,477 INFO worker-2 started
2017-11-12 10:19:50,478 INFO worker-0 got connect 2
2017-11-12 10:19:50,478 INFO worker-3 started
2017-11-12 10:19:50,479 INFO worker-4 started
2017-11-12 10:19:50,480 INFO worker-1 got connect 1
2017-11-12 10:19:50,481 INFO worker-2 got connect 0
2017-11-12 10:19:51,486 INFO worker-0 return resource 2
2017-11-12 10:19:51,486 INFO worker-1 return resource 1
2017-11-12 10:19:51,486 INFO worker-3 got connect 2
2017-11-12 10:19:51,487 INFO worker-4 got connect 1
2017-11-12 10:19:52,490 INFO worker-2 return resource 0
2017-11-12 10:19:52,491 INFO worker-3 return resource 2
2017-11-12 10:19:52,492 INFO worker-4 return resource 1


## 线程之间的通讯

In [45]:
import queue

In [46]:
def producer(queue: queue.Queue, event: threading.Event):
    while not event.wait(3):
        data = random.randint(0, 100)
        logging.info(data)
        queue.put(data)
        
def consumer(queue: queue.Queue, event: threading.Event):
    while not event.is_set():
        logging.info(queue.get())

In [47]:
q = queue.Queue()

In [48]:
e = threading.Event()

In [49]:
threading.Thread(target=consumer, args=(q, e), name='consumer').start()

In [50]:
threading.Thread(target=producer, args=(q, e), name='producer').start()

2017-11-12 10:57:50,869 INFO producer 99
2017-11-12 10:57:50,870 INFO consumer 99
2017-11-12 10:57:53,874 INFO producer 87
2017-11-12 10:57:53,875 INFO consumer 87
2017-11-12 10:57:56,880 INFO producer 99
2017-11-12 10:57:56,881 INFO consumer 99
2017-11-12 10:57:59,886 INFO producer 90
2017-11-12 10:57:59,887 INFO consumer 90
2017-11-12 10:58:02,888 INFO producer 60
2017-11-12 10:58:02,889 INFO consumer 60
2017-11-12 10:58:05,893 INFO producer 92
2017-11-12 10:58:05,895 INFO consumer 92
2017-11-12 10:58:08,899 INFO producer 69
2017-11-12 10:58:08,899 INFO consumer 69
2017-11-12 10:58:11,900 INFO producer 85
2017-11-12 10:58:11,901 INFO consumer 85
2017-11-12 10:58:14,906 INFO producer 19
2017-11-12 10:58:14,907 INFO consumer 19
2017-11-12 10:58:17,911 INFO producer 60
2017-11-12 10:58:17,913 INFO consumer 60


In [51]:
e.set()

In [52]:
queue.Queue # 先进先出的 队列

queue.Queue

In [53]:
queue.PriorityQueue

queue.PriorityQueue

In [55]:
q = queue.Queue(maxsize=5)

In [56]:
for x in range(5):
    q.put(x)

In [57]:
q.qsize()

5

In [58]:
q.put(6)

KeyboardInterrupt: 

In [59]:
q.put(6, block=False)

Full: 

In [60]:
q.put(6, timeout=3)

Full: 

In [61]:
q.put_nowait(6) # q.put(6, block=False)

Full: 

In [62]:
q.get()

0

In [63]:
q.qsize()

4

In [68]:
q.get(block=False)

Empty: 

In [69]:
q.get()

KeyboardInterrupt: 

In [70]:
q.get(timeout=3)

Empty: 

In [72]:
q.get_nowait(3) # q.get(3, block=False)

TypeError: get_nowait() takes 1 positional argument but 2 were given

In [75]:
q.not_full()

TypeError: 'Condition' object is not callable

In [76]:
import collections

In [ ]:
collections.

## 多进程

In [77]:
import multiprocessing

In [78]:
def worker():
    logging.info('worker')

In [79]:
p = multiprocessing.Process(target=worker)

In [80]:
p.start()

2017-11-12 11:10:58,238 INFO MainThread worker


In [81]:
importlib.reload(logging)
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(levelname)s %(threadName)s [%(processName)s] %(message)s')

In [83]:
p = multiprocessing.Process(target=worker, name='worker')
p.start()

2017-11-12 11:12:03,587 INFO MainThread [worker] worker


In [84]:
p.pid

8847

In [85]:
p.exitcode

0

In [86]:
p.terminate()

In [87]:
p.pid

8847

In [88]:
multiprocessing.current_process

<function multiprocessing.process.current_process>

In [ ]:
multiprocessing.Semaphore

数据的序列化和反序列化吗？

In [89]:
multiprocessing.Queue

<bound method BaseContext.Queue of <multiprocessing.context.DefaultContext object at 0x10d069668>>

In [90]:
from multiprocessing import Manager

In [91]:
mgr = Manager()

In [92]:
d = mgr.dict()

In [93]:
d

<DictProxy object, typeid 'dict' at 0x10be8dbe0>

In [94]:
lst = mgr.list()

In [95]:
lst

<ListProxy object, typeid 'list' at 0x10d682710>

In [96]:
ns = mgr.Namespace()

In [97]:
ns.f = 3

In [98]:
ns

<NamespaceProxy object, typeid 'Namespace' at 0x10d3549b0>

In [99]:
ns.f

3

In [ ]:
ns.data = pickle.dumps('sadfsdf')

In [100]:
pickle.loads(ns.data)

NameError: name 'pickle' is not defined

In [101]:
nginx

NameError: name 'nginx' is not defined

## concurrent

In [102]:
import concurrent

In [103]:
from concurrent import futures

In [104]:
futures.ThreadPoolExecutor

concurrent.futures.thread.ThreadPoolExecutor

In [105]:
pool = futures.ThreadPoolExecutor(max_workers=5)

In [106]:
pool.submit

<bound method ThreadPoolExecutor.submit of <concurrent.futures.thread.ThreadPoolExecutor object at 0x10dded390>>

In [107]:
fut = pool.submit(lambda: 1 + 1)

In [111]:
fut.result()

2

In [112]:
fut.done()

True

In [113]:
def worker():
    time.sleep(30)
    logging.info('worker')

In [114]:
fut = pool.submit(worker)


In [115]:
fut.done()

False

In [116]:
fut.cancel()

False

In [117]:
fut.done()

False

2017-11-12 11:31:28,096 INFO <concurrent.futures.thread.ThreadPoolExecutor object at 0x10dded390>_0 [MainProcess] worker


In [118]:
def worker():
    raise Exception('haha')

In [119]:
fut = pool.submit(worker)

In [120]:
fut.exception()

Exception('haha')

In [122]:
help(pool.submit) # = threading.Thread(target=fn, agrs=args, kwargs=kwargs)

Help on method submit in module concurrent.futures.thread:

submit(fn, *args, **kwargs) method of concurrent.futures.thread.ThreadPoolExecutor instance
    Submits a callable to be executed with the given arguments.
    
    Schedules the callable to be executed as fn(*args, **kwargs) and returns
    a Future instance representing the execution of the callable.
    
    Returns:
        A Future representing the given call.



In [123]:
futures.ProcessPoolExecutor

concurrent.futures.process.ProcessPoolExecutor

事实上，在python3中，确实建议大家使用futures 模块 待代替 threading multiprocessing

In [ ]:
threading.Thread.daemon

In [125]:
futures._base.Executor

concurrent.futures._base.Executor

- 线程通讯：使用 queue 内置容器
- 进程通讯：multiprocessing.Queue, Manager
- 阻塞/非阻塞
- 同步/异步

下周  就讲网络编程